In [1]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
from pdfminer.layout import LAParams
# To extract text from tables in PDF
import pdfplumber
#file management
import os
import sys
import re
#other
import numpy as np
from itertools import chain
from collections import Counter
from pprint import pprint

The code adapts to the post: https://towardsdatascience.com/extracting-text-from-pdf-files-with-python-a-comprehensive-guide-9fc4003d517

In [2]:
print('current directory: ' + os.getcwd())
data_path='../data/demo_data/'
files = [f for f in os.listdir(data_path) if f.endswith('.pdf')]
print('demofiles:')
for file in files:
    print(file)

current directory: c:\Users\linda.hemmann\Documents\FullTextTagging\notebooks
demofiles:
1-s2.0-S0090825815002577-main.pdf
1-s2.0-S014341799790086X-main.pdf
1-s2.0-S0167814004005663-main.pdf
1-s2.0-S1525730414002630-main.pdf
paper0.pdf
paper1.pdf
paper2.pdf
paper4.pdf
PII_ S0196-9781(97)00323-9.pdf
PII_ S1368-8375(98)00013-X.pdf


Text extraction

## Fully pyPDF2 approach

In [3]:
pdf_path = data_path+files[1]
# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}

In [4]:
text = ''
#simple approach seems to work better regarding the order of text
for i in range(len(pdfReaded.pages)):
    page = pdfReaded.pages[i]
    text += page.extract_text()
#print(text)

In [5]:
reader = PyPDF2.PdfReader(pdf_path)
reader.metadata
len(reader.pages)

7

In [6]:
#code from : https://gist.github.com/tiarno/8a2995e70cee42f01e79
def walk(obj, fnt, emb):
    '''
    If there is a key called 'BaseFont', that is a font that is used in the document.
    If there is a key called 'FontName' and another key in the same dictionary object
    that is called 'FontFilex' (where x is null, 2, or 3), then that fontname is 
    embedded.
    
    We create and add to two sets, fnt = fonts used and emb = fonts embedded.
    '''
    if not hasattr(obj, 'keys'):
        return None, None
    fontkeys = set(['/FontFile', '/FontFile2', '/FontFile3'])
    if '/BaseFont' in obj:
        fnt.add(obj['/BaseFont'])
    if '/FontName' in obj:
        if [x for x in fontkeys if x in obj]:# test to see if there is FontFile
            emb.add(obj['/FontName'])

    for k in obj.keys():
        walk(obj[k], fnt, emb)

    return fnt, emb# return the sets for each page

if __name__ == '__main__':
    fname = pdf_path
    pdf = pdfReaded #PdfFileReader(fname)
    fonts = set()
    fonts_list =[]
    embedded = set()
    for page in pdf.pages:
        obj = page.get_object()
        # updated via this answer:
        # https://stackoverflow.com/questions/60876103/use-pypdf2-to-detect-non-embedded-fonts-in-pdf-file-generated-by-google-docs/60895334#60895334 
        # in order to handle lists inside objects. Thanks misingnoglic !
        # untested code since I don't have such a PDF to play with.
        if type(obj) == PyPDF2.generic.ArrayObject:  # You can also do ducktyping here
            for i in obj:
                if hasattr(i, 'keys'):
                    f, e = walk(i, fonts, embedded_fonts)
                    fonts = fonts.union(f)
                    fonts_list.append(f)
                    embedded = embedded.union(e)
        else:
            f, e = walk(obj['/Resources'], fonts, embedded)
            fonts = fonts.union(f)
            fonts_list.append(f)
            embedded = embedded.union(e)

    unembedded = fonts - embedded
    print('Font List')
    pprint(sorted(list(fonts)))
    if unembedded:
        print('\nUnembedded Fonts')
        pprint(unembedded)
    print(fonts_list)

Font List
['/Arial',
 '/Arial,Bold',
 '/Arial,Italic',
 '/TimesNewRoman',
 '/TimesNewRoman,Bold',
 '/TimesNewRoman,Italic']

Unembedded Fonts
{'/Arial',
 '/Arial,Bold',
 '/Arial,Italic',
 '/TimesNewRoman',
 '/TimesNewRoman,Bold',
 '/TimesNewRoman,Italic'}
[{'/TimesNewRoman', '/TimesNewRoman,Italic', '/Arial,Italic', '/Arial,Bold', '/TimesNewRoman,Bold', '/Arial'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial', '/TimesNewRoman,Italic'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial', '/TimesNewRoman,Italic'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial', '/TimesNewRoman,Italic'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial', '/TimesNewRoman,Italic'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial', '/TimesNewRoman,Italic'}, {'/Arial,Italic', '/Arial,Bold', '/TimesNewRoman', '/TimesNewRoman,Bold', '/Arial',

## pfdMiner + pyPDF2 approach

In [7]:
# Create a function to extract text

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        char_formats= []
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character and the size
                    char_formats.append((character.fontname,round(character.size)))
                    # Append the font size of the character
                    #line_formats.append(character.size)
    
        # Find the unique font sizes and names in the line
        if char_formats:
            #test = Counter(chain(line_formats))
            #values, counts = np.unique(line_formats, return_counts=True)
            #format_per_line = test.most_common(1)
            #[values[np.argmax(counts)], counts[np.argmax(counts)]]
            line_formats.append(max(set(char_formats), key=char_formats.count)) #list(set(line_formats))
        else:
            format_per_line=[]
    
    # Return a tuple with the text in each line along with its format
    return (line_text, line_formats)#format_per_line)

In [8]:
def remove_header_footer(pdf_extracted_text):
        page_format_pattern = r'([page]+[\d]+)'
        #pdf_extracted_text = pdf_extracted_text.lower().split("\n")
        header = pdf_extracted_text[0].strip().lower()
        footer = pdf_extracted_text[-1].strip().lower()
        if re.search(page_format_pattern, header) or header.isnumeric():
            pdf_extracted_text = pdf_extracted_text[1:]
        if re.search(page_format_pattern, footer) or footer.isnumeric():
            pdf_extracted_text = pdf_extracted_text[:-1]
        pdf_extracted_text = "\n".join(pdf_extracted_text)
        return pdf_extracted_text

In [9]:
pdf_path = data_path+files[1]
# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}

In [10]:
#fine tune the pdfMiner parameters, https://pdfminersix.readthedocs.io/en/latest/reference/composable.html#laparams
laparams = LAParams(line_margin = 0.6, char_margin = 3, boxes_flow = -0.6, word_margin = 0.2 )# line_overlap = 0.1)
all_formats = []
for pagenum, page in enumerate(extract_pages(pdf_path, laparams=laparams)):
    # Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()


    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page 
    #page_elements.sort(key=lambda a: a[0], reverse=True)

    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]
        
        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass
    print(i)
    #exclude headers and page numbers
    #page_content = remove_header_footer(page_content)
    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    all_formats.append(line_format)
    # Add the list of list as the value of the page key
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Closing the pdf file object
pdfFileObj.close()

# Deleting the additional files created
#os.remove('cropped_image.pdf')
#os.remove('PDF_image.png')

# Display the content of the page
result = '\n \n'.join(text_per_page['Page_0'][4])
print(result)
print(text_per_page['Page_0'][1])



14
16
19
12
31
59
9
Bombesin.like  peptide  is present  in 
duct  cells  in salivary  glands:  studies 
on  normal  and  irradiated  animals 

 
Y. Aalto, U. Kj6rell, R. Henriksson, L. Franzen, S. Forsgren 

 
Departments  of Anatomy and Oncology,  Ume& University, S-901 87 Ume&, Sweden 

 
Summary  Bombesin (BN) and its mammalian  counterpart gastrin-releasing peptide act as neuroregulatory  hormones 
and tissue-specific growth factors, and have been implicated as peripheral and central  satiety-inducing agents.  In the 
present study, the immunohistochemical  expression of BN in submandibular, sublingual and parotid glands of rats was 
examined 10 days after 5 consecutive days with daily doses of 6-8 Gy irradiation. Radioimmunoassay (RIA)  methods 
were also  used.  Immunoreactive  granular  structures were observed within duct cells of both  controls  and  irradiated 
animals.  In  the  parenchyma  of  irradiated  animals,  very  few  nerve  fibres  showing  BN-like  immunoreactivit

Find the most common font

In [11]:
flat_list = []
for page in text_per_page:
    for block in text_per_page[page][1]:
        for item in block:
            flat_list.append(item)
#flat_list = [text_per_page[p][1] for p in text_per_page]
#test = Counter(chain(line_formats))
        #values, counts = np.unique(line_formats, return_counts=True)
#        format_per_line = test.most_common(1)

In [12]:
test = Counter(chain(flat_list))
most_common_format = test.most_common(1)
print(most_common_format)
most_common_format = most_common_format[0][0]

[(('Times-Roman', 10), 303)]


In [13]:
#flat_list = [item for sublist in all_formats for item in sublist]
#most_common_format = max(flat_list, key=flat_list.count)
print(most_common_format)

('Times-Roman', 10)


In [14]:
main_text=''
i=0
for page in text_per_page:
    print('PAGE: ' + str(page))
    for format, text in zip(text_per_page[page][1],text_per_page[page][0]):
        #print(format)
        #print(text)
        if most_common_format in format:
            main_t = text
            print(main_t)
        #main_t = [block[0][idx] for idx in range(len(block[0])) if block[1][idx]==most_common_format]
        #print(main_t)
        #print( [ block[1][idx]==most_common_format for idx in range(len(block[0]))])
        #i+=1

PAGE: Page_0
Bombesin (BN) is a tetradecapeptide initially purified from 
the skin of the frog Bombina bombina. 1,2 One mammalian 
homologue  of bombesin  is  the  27-amino  acid  peptide, 
gastrin-releasing  peptide  (GRP),  isolated  from  porcine 
stomach. 3 Immunoreactivity  for  BN  or  GRP  has  been 
detected  in  the  nervous  system 4  as  well  as  in 
macrophages 5 and  neuroendocrine  cellsY BN/GRP acts 
as  neurotransmitter,  paracrine  hormone  and  tissue- 
specific growth factor in normal and neoplastic tissues. 8-~1 
Furthermore,  BN/GRP  is  a  potent  satiety  agent,  being 
implicated in the physiological regulation of food intake.~2 
We have previously demonstrated that irradiation leads 
to an increase in the immunohistochemical expression of 
BN-like  peptide  in  the  innervation  in  submandibular 

glands ~3 and submucosal glands in the larynx ~4 of the rat. 
These observations may be of great importance since BN, 
as  discussed  above, functions as a  neurore

In [15]:
main_text=''
i=0
for page in text_per_page:
    p = text_per_page[page]
    main_t = [p[0][idx] for idx in range(len(p[0])) if p[1][idx]==most_common_format]
    print(main_t)
    print( [ p[1][idx]==most_common_format for idx in range(len(p))])
    i+=1
    #main_text=main_text.join( p[0][p[1]==most_common_format])

[]
[False, False, False, False, False]
[]
[False, False, False, False, False]
[]
[False, False, False, False, False]
[]
[False, False, False, False, False]
[]
[False, False, False, False, False]
[]
[False, False, False, False, False]
[]
[False, False, False, False, False]


In [17]:
print(main_text)